<a href="https://colab.research.google.com/github/COOLSTORM36/EncryptEverything/blob/main/EncryptEverything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EncryptEverything

## I'm going to use the cryptography as the library

In [ ]:
!pip install cryptography

## Prerequisite

In [19]:
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import secrets
import os

from cryptography.hazmat.primitives.ciphers.algorithms import AES256

## Generate the custom key from password

In [81]:
def generate_key_from_password(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
    )

    key = kdf.derive(password.encode())

    return key

## Generate random password

In [61]:
def generate_random_password():
    length = 16
    password = secrets.token_urlsafe(length)
    print("Generated a new password, please keep it safe(only shown once):", password)

    return password

## Method Block(Fernet)

### Fernet encrypt

In [80]:
# Encrypt file using Fernet
def fernet_encrypt_file(file_path, password=None):
    # Read the data from the file
    try:
        with open(file_path, "rb") as file:
            file_data = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return

    # Detect if user input password
    if not password:
        password = generate_random_password()

    # Generate a random salt
    salt = os.urandom(16)
    key = base64.urlsafe_b64encode(generate_key_from_password(password, salt))

    # Get the original file extension
    file_path_no_ext, file_extension = os.path.splitext(file_path)

    # Create the key to encrypt the file
    f = Fernet(key)
    encrypted_data = f.encrypt(file_data)
    encrypted_extension = f.encrypt(file_extension.encode())

    # Create the encrpted file in the same direcotry
    encrypted_file_content = salt + encrypted_extension + b"||" + encrypted_data
    encrypted_file_path = file_path_no_ext + '.ee'

    with open(encrypted_file_path, "wb") as encrypted_file:
        encrypted_file.write(encrypted_file_content)

    print(f"File encrypted successfully to {encrypted_file_path}")

    return None

### Fernet decrypt

In [87]:
def fernet_decrypt_file(file_path, password):
    # Open the encrypted file
    try:
        with open(file_path, "rb") as encrypted_file:
            encrypted_file_content = encrypted_file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return

    # Detect if file is deprecated
    try:
        # Fetch the Salt, Extension, and Original Contents
        salt = encrypted_file_content[:16]
        remining_content = encrypted_file_content[16:]
        encrypted_extension, encrypted_data = remining_content.split(b"||", 1)

        key = base64.urlsafe_b64encode(generate_key_from_password(password, salt))
        # Decrypt the data
        f = Fernet(key)
        file_extension = f.decrypt(encrypted_extension).decode()
        decrypted_data = f.decrypt(encrypted_data)

        # Save the data
        decrypted_file_path = os.path.splitext("/content/sample_data/README.ee")[0] + file_extension
        with open(decrypted_file_path, "wb") as decrypted_file:
            decrypted_file.write(decrypted_data)

        print(f"File decrypted successfully to {decrypted_file_path}")

        return None

    except:
        return "Invalid Password or Deprecated File"

In [85]:
fernet_encrypt_file("/content/sample_data/README.md", "password")

File encrypted successfully to /content/sample_data/README.ee


In [88]:
fernet_decrypt_file('/content/sample_data/README.ee',"password")

File decrypted successfully to /content/sample_data/README.md


## Method block(AES-256)

## AES-256 Encrypt

In [91]:
def aes_256_encrypt_file(file_path, password=None):
    # Read the data from the file
    try:
        with open(file_path, "rb") as file:
            file_data = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return

    # Detect user if input password
    if not password:
        password = generate_random_password()

    # Generate a random salt and nonce
    salt = os.urandom(16)
    nonce = os.urandom(12)

    key = generate_key_from_password(password, salt)

    # Create AES cipher in GCM mode
    cipher = Cipher(algorithms.AES(key), modes.GCM(nonce))
    encryptor = cipher.encryptor()

    # Encrypt the data
    cipher_text = encryptor.update(file_data) + encryptor.finalize()

    print(len(cipher_text))

    return None

## AES-256 Decrypt

In [90]:
aes_256_encrypt_file("/content/sample_data/README.md")

Generated a new password, please keep it safe(only shown once): FwhYayn_fC4iN4Xph3TNtw
32
962
